In [ ]:
#install.packages("httr2", repos='http://cran.us.r-project.org')

In [1]:
library(readxl)
library(dplyr)
library(stringr)
library(httr2)
library(jsonlite)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
# using the processed file so some of the following steps are redundant
# should be replaced with the Pure export
osfDF <- read_xlsx("../data/osf_open_data.xlsx")

In [3]:
# select only cases where there is (or likely is) public data
openData <- osfDF %>% filter(Public.data == "Yes" | Public.data == "Maybe")

In [4]:
# clean up the names
# NOTE we should attempt fuzzy matching so let's not manually clean the names for now
#openData %>% mutate(Name2 = str_remove_all(str_to_lower(Name), "prof. dr. |dr. |drs. |ing. |, msc|, mba|, ba bsc"))

In [5]:
# extract raw osf ids
ids <- str_extract(openData$Osf.id, "(https://osf.io/)(.*)/", 2) 

In [6]:
# deduplicate
ids <- unique(ids)

In [7]:
# since we will call the api many times it is best to define a function
fetch_results <- function(url) {

    # make the request
    response <- request(url) %>% req_perform()
    # no need to check for server-side errors, httr2 turns them into R errors
    results <- response %>% resp_body_json()
    
    return(results)
}

In [8]:
datasets <- data.frame()  # here we store user and dataset ids

for (id in ids) {

    url <- paste0("https://api.osf.io/v2/users/", id, "/nodes/")
    
    # pagination: continue until no more next pages left (i.e. until url is NULL)
    while (!is.null(url)) {
        # fetch results from page
        results <- fetch_results(url)
        # find all datasets
        for (node in results$data) {
            if (node$attributes$category == "data") {
                datasets <- rbind(datasets, list(id, node$id))
            }
        }
        # link to next page is new url
        url <- results$links$'next'
    }
}

In [9]:
colnames(datasets) <- c("user.id", "dataset.id")